# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Giorgio Maria Di Nunzio
#### (Courtesy of Gabriele Tolomei FIS 2018-2019)
<a href="mailto:giorgiomaria.dinunzio@unipd.it">giorgiomaria.dinunzio@unipd.it</a><br/>
University of Padua, Italy<br/>
2021/2022<br/>

# Lecture 9: Data Preparation with <code>pandas</code>

## What Does Data Preparation Mean?

-  It typically consists of **loading**, **cleaning**, **transforming**, and **rearranging** data (the last three steps are also referred to as data **"munging"**). 

-  Such tasks are often reported to take up 80% or more of your development time (for a machine learning/data science task). 

-  Sometimes it can be achieved by using a mixture of tools, i.e., from general-purpose programming languages, like Python, Perl, R, or Java, to UNIX tools like <code>**sed**</code> or <code>**awk**</code>.

-  Luckily, <code>**pandas**</code> provides you with a single, high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

## Our Focus

1.  Handling missing data (**NA** or **N**ot **A**vailable)

2.  Dealing with duplicates

3.  Managing very extreme values (i.e., **outliers**)

4.  Combining multiple datasets into a single one

In [ ]:
"""
Before we start our journey on data preparation with pandas,
we get back to the example we used in Lecture 08.
"""
import numpy as np
import pandas as pd

url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user'
# The first line of the file represents the header, and each field
# is separated by a pipe
"""
We specify the url where the data is located, the character u|sed to separate fields ('|')
and the name of the column to use as row label (otherwise, RangeInteger will be used)
"""
users = pd.read_csv(url, sep='|', index_col='user_id')
print(users.head())

## Handling Missing Data (*NA*)

-  Missing data (**NA**) may either be data that does not exist or that exists but was not observed (e.g., due to measurement issues).

-  <code>**pandas**</code> makes working with missing data as painless as possible. For example, all of the descriptive statistics on <code>**pandas**</code> objects exclude **NA** by default.

-  <code>**pandas**</code> represents **NA** using so-called **sentinel values**.

-  Two of the most common sentinel values are <code>**None**</code> and the floating point value <code>**NaN**</code> (**N**ot **a** **N**umber).

In [ ]:
"""
Let's see if there are any missing data (NA) on our loaded dataset.
"""

# isnull() returns a boolean DataFrame with the same shape of the DataFrame object
# where you invoke the method. Each entry of this new boolean DataFrame either contains
# True or False depending on whether the corresponding entry in the original DataFrame
# is missing or not
print("Shape of the boolean DataFrame: {}\n".format(users.isnull().shape))
print(users.isnull().head())

In [ ]:
"""
In order to see which column has at least one missing value,
we can call the method any() on the boolean DataFrame above.
This returns a boolean Series which contains an entry for each column 
(row aggregation) which evaluates to True if at least one element
of that column is True, False otherwise.
"""
print("Shape of the boolean Series: {}\n".format(users.isnull().any().shape))
print(users.isnull().any().head())

In [ ]:
"""
We have verified that our dataset does not contain any missing value.
What if, though, I would like to obtain a row-wise (i.e., column aggregation)
boolean Series containing a value for each row, which tells me whether that
row contains or not at least one NA value?
"""

print("Shape of the boolean Series: {}\n".format(users.isnull().any(axis=1).shape))
print(users.isnull().any(axis=1).head())

In [ ]:
"""
In order to find whether there exists at least one NA on the whole DataFrame
we can simply aggregate one more time the boolean Series above using another any()
"""

print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(users.isnull().any().any()))

In [ ]:
"""
Of course, the same thing can be achieved using different approaches.
This is possibly the quickest solution.
"""

# values returns a 2-D numpy array (ndarray) and any() "flatten" it 
print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(users.isnull().values.any()))

In [ ]:
"""
This solution makes exactly the same 3 steps as above but instead of computing
boolean aggregation from the boolean DataFrame, it sum boolean values over the rows
(i.e., column-wise) and finally makes a final aggregation step (sum) to obtain
the number of missing values of the DataFrame.
NOTE: this is generally slower but it provides you with an extra information
(i.e., how many NA values are in the DataFrame, not just a boolean value!)
"""
print("There are {} missing values (NA) in the DataFrame".
      format(users.isnull().sum().sum()))

In [ ]:
"""
Suppose we want to randomly perturbate our dataset with some missing values.
First of all, let's create a deep copy of our original DataFrame.
"""

# Prepare the deep copy of the DataFrame where we are going to randomly insert
# some missing values (NA)
users_with_na = users.copy()

In [ ]:
"""
Let's create a uniform random sample of size=10 drawn from the range [1, 943]
"""
np.random.seed(42)
row_indices = np.random.randint(low = 1, high = 944, size = 10)
# alternatively, use np.random.choice(np.arange(1, 944), size=10, replacement=False)
print("Random row indices: {}".format(row_indices))

In [ ]:
"""
We are going to use the first half of the randomly selected indices
for populating with np.NaN the column 'age', whilst the second half
is going to be used to set to None the column 'occupation'.
The two middle indices will be used for both. 
In other words:
- rows labeled as 103, 436, 861, 271 will be used to
set column 'age' to np.NaN
- rows labeled as 701, 21, 615, 122 will be used to
set column 'occupation' as None
- rows labeled as 107 and 72 will be used to set both
column 'age' to np.NaN and column 'occupation' to None
"""

In [ ]:
print("Let's first extract the records we want to update for column 'age':\n{}".
      format(users_with_na.loc[row_indices[:6], 'age']))

In [ ]:
print("Let's first extract the records we want to update for column 'age':\n{}".
      format(users_with_na.loc[row_indices[:6], 'age']))

In [ ]:
print("Let's now extract the records we want to update for column 'occupation':\n{}".
      format(users_with_na.loc[row_indices[4:], 'occupation']))
# I swear, I didn't do it on purpose! 
# This was truly the outcome of a purely (pseudo-)random experiment but apparently
# 'scientist' will be sacrificed and set to None

In [ ]:
"""
Let's set the values as we planned.
"""
users_with_na.loc[row_indices[:6], 'age'] = np.nan
users_with_na.loc[row_indices[4:], 'occupation'] = None
# NOTE: the same won't work if we use something like the following:
# users_with_na.loc[row_indices[:6]]['age'] = np.nan
# users_with_na.loc[row_indices[4:]]['occupation'] = None
# This is because in case of [] operator, we are actually 
# accessing a copy of the selected slice, whereas above we are working on a view.
# More information on this - a.k.a. SettingWithCopyWarning - is available here: 
# https://www.dataquest.io/blog/settingwithcopywarning/

In [ ]:
"""
Let's verify if changes actually took place!
"""
print("Let's see how column 'age' looks like:\n{}".
      format(users_with_na.loc[row_indices[:7], 'age']))
print()
print("Let's see how column 'occupation' looks like:\n{}".
      format(users_with_na.loc[row_indices[3:], 'occupation']))

In [ ]:
"""
Now, let's try to see if the tests above we run for finding any NA work correctly.
"""
print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(users_with_na.isnull().any().any()))
print()
print("There are {} missing values (NA) in the DataFrame".
      format(users_with_na.isnull().sum().sum()))

## Filtering Missing Data

In [ ]:
"""
You may want to drop rows or columns of a DataFrame which are all NA 
or only those containing any NAs. 
The pandas.dropna function by default drops any row containing a missing value.
By default, dropna returns a new object but we can specify inplace=True for
any in-place change.
"""

# This will drop all the rows containing at least one NA
cleaned_users = users_with_na.dropna()

# After issuing the above command, we are expecting the DataFrame to have 10 rows less
# (i.e., 933 instead of 943)
print("Number of records in the original DataFrame = {}\n".format(users_with_na.shape[0]))
print("Number of records in the cleaned DataFrame = {}".format(cleaned_users.shape[0]))

In [ ]:
"""
If we instead want to delete only those rows which are FULL of NAs,
then we have to pass how='all' to dropna.
"""
# This will drop all the rows containing ALL NAs
cleaned_users_all = users_with_na.dropna(how = 'all')

# After issuing the above command, how many rows will be dropped?
print("Number of records in the original DataFrame = {}\n".format(users_with_na.shape[0]))
print("Number of records in the cleaned DataFrame = {}".format(cleaned_users_all.shape[0]))

In [ ]:
"""
As usual, dropna works on rows (axis 0) by default.
Instead, if we want to delete columns corresponding to a missing value
we can pass axis=1 argument to dropna.
"""
# This will drop all the columns containing at least one NA
cleaned_users_columns = users_with_na.dropna(axis = 1)

In [ ]:
# How many columns are we expecting the above command will drop?

In [ ]:
# After issuing the above command, weare expecting the DataFrame to have 2 columns less
# (i.e., 2 instead of 4, as 'age' and 'occupation' have both at least one NA value)
print("Number of columns in the original DataFrame = {}\n".format(users_with_na.shape[1]))
print("Number of columns in the cleaned DataFrame = {}".format(cleaned_users_columns.shape[1]))

## Filling in Missing Data

In [ ]:
"""
'fillna' fill in missing data with some values, rather than filtering them out.
"""
# Suppose we fill all missing values with 0 (not in-place, otherwise set inplace=True)
users_fill_na = users_with_na.fillna(0)

# Let's verify if changes actually took place!
print("Let's see how column 'age' looks like:\n{}".
      format(users_fill_na.loc[row_indices[:6], 'age']))
print()
print("Let's see how column 'occupation' looks like:\n{}".
      format(users_fill_na.loc[row_indices[4:], 'occupation']))

In [ ]:
"""
We can also pass a dictionary to fillna in order to specify
different values to replace NA with.
"""
users_fill_na = users_with_na.fillna({'age': 0, 'occupation': 'none'})
# Let's verify if changes actually took place!
print("Let's see how column 'age' looks like:\n{}".
      format(users_fill_na.loc[row_indices[:6], 'age']))
print()
print("Let's see how column 'occupation' looks like:\n{}".
      format(users_fill_na.loc[row_indices[4:], 'occupation']))

## Removing Duplicates

In [ ]:
"""
The DataFrame method duplicated() returns a boolean Series 
indicating whether each row is a duplicate or not.
"""
# Let's go back to our original DataFrame
print("Duplicated rows:\n{}".format(users.duplicated().head()))
print()

In [ ]:
# What if I would like to see if there exists at least one duplicated row?
print("Q: Is there at least one duplicated row? A: {}"
      .format(users.duplicated().any()))

In [ ]:
# Suppose I want to extract only the duplicated rows
print("Duplicated rows:\n{}".format(users))

In [ ]:
users.duplicated()

In [ ]:
# Suppose I want to extract only the duplicated rows
print("Duplicated rows:\n{}".format(users[users.duplicated()]))

In [ ]:
"""
By default, if you have 3 duplicated rows, only the last two will be
marked as duplicates (i.e., the first occurrence is kept).
This can be changed by specifying the parameters 'keep'
keep : {'first', 'last', False}, default 'first'
first : Mark duplicates as True except for the first occurrence.
last : Mark duplicates as True except for the last occurrence.
False : Mark all duplicates as True.
"""
# Suppose I want to extract only the duplicated rows, this time considering them all
print("Duplicated rows:\n{}".format(users[users.duplicated(keep = False)]))

In [ ]:
"""
Relatedly, drop_duplicates() returns a DataFrame where the duplicated array is False.
"""
# Remove duplicated rows (keeping the first occurrence of each duplicates)
users_with_no_dup = users.drop_duplicates()
# What if I would like to see if there exists at least one duplicate row, now?
print("Q: Is there at least one duplicated row? A: {}"
      .format(users_with_no_dup.duplicated().any()))
print()
print("Total number of rows after removing duplicated rows = {}"
      .format(users_with_no_dup.shape[0]))

In [ ]:
"""
We can also specify the same 'keep' argument to decide on how to
mark duplicates, and therefore remove them
"""
# Remove ALL duplicated rows
users_with_no_dup = users.drop_duplicates(keep = False)
# What if I would like to see if there exists at least one duplicate row, now?
print("Q: Is there at least one duplicated row? A: {}"
      .format(users_with_no_dup.duplicated().any()))
print()
print("Total number of rows after removing duplicated rows = {}"
      .format(users_with_no_dup.shape[0]))

In [ ]:
"""
By default, both duplicated() and drop_duplicates() consider all of the columns; 
alternatively we can specify any subset of them to detect duplicates. 
Suppose we want to filter duplicates only based on the 'gender' and 'occupation' columns.
"""
# Suppose I want to extract only the duplicated rows w.r.t. 'gender' and 'occupation'
print("There are {} duplicated rows having the same 'gender' and 'occupation'.\n\
The following are the first 5 of them:\n{}"
      .format(users[users.duplicated(['gender', 'occupation'])].shape[0],
              users[users.duplicated(['gender', 'occupation'])].head()))

## Transforming Data Using a Function or Mapping

In [ ]:
"""
For many data sets, we may wish to perform some transformation based on the values
in an array, Series, or column in a DataFrame.
"""
# Suppose we want to add a column indicating the salary for each occupation. 
# Let's write down a mapping of each occupation to salary
occupation_to_salary = {'technician': 25000, 'administrator': 150000,
                        'writer': 40000, 'executive': 300000, 'other': 18000,
                        'student': 1300, 'lawyer': 27500, 'educator': 45000,
                        'scientist': 60000, 'entertainment': 185000, 'programmer': 55000,
                        'librarian': 22000, 'homemaker': 240000, 'artist': 72000,
                        'engineer': 91000, 'marketing': 66000, 'none': 0,
                        'healthcare': 41000, 'retired': 52000, 'salesman': 48000,
                        'doctor': 140000
                       }

In [ ]:
occupation_to_salary

In [ ]:
users['occupation'].map(occupation_to_salary)

In [ ]:
"""
The map method on a Series accepts a function or dict-like object containing a mapping.
"""
users['salary'] = users['occupation'].map(occupation_to_salary)
print(users.head())

In [ ]:
"""
Alternatively, we could also have passed to map a lambda function that does all the work.
"""
# Let's first delete the salary column
del users['salary']
# Verify the column is really removed
print(users.head())

In [ ]:
# lambda function
users['salary'] = users['occupation'].map(lambda x: occupation_to_salary[x])
print(users.head())

In [ ]:
# lambda function
users['salary'] = users['occupation'].map(lambda o: occupation_to_salary[o])
print(users.head())

## Replacing Values

In [ ]:
"""
Filling in missing data with the 'fillna' method is a special case 
of more general value replacement. 
While 'map' can be used to modify a subset of values in an object, 
'replace' provides a simpler and more flexible way to do so.
"""
# Suppose we would like to consider 'none' value of 'occupation' as missing data
# (i.e., 'none' can be considered as a sentinel value)
# NOTE: remember that this can be also specified when loading the DataFrame
# by specifying na_values
users['occupation'] = users['occupation'].replace('none', np.nan)
print("The number of replaced rows is = {}"
      .format(users[(users['occupation'].isnull())].shape[0]))
print("The following are the first 5 rows that have been replaced:\n{}"
      .format(users[(users['occupation'].isnull())].head()))

In [ ]:
# Let's verify that 'occupation' contains actually some NaN
print("Q: Is there any missing value for 'occupation'? A: {}".
      format(users['occupation'].isnull().any()))

In [ ]:
# Suppose I want to update the salary column corresponding to
# those rows where 'occupation' is now missing
mask = users['occupation'].notnull()
# Update 'salary' where 'occupation' is null. The semantics is as follows.
# 'salary' will keep its value if the mask condition is verified
# (i.e., if 'occupation' is NOT null, otherwise we set it to NaN)
users['salary'] = users['salary'].where(mask, np.nan)
#np.where(mask, users.salary, np.nan)

In [ ]:
print("The number of replaced rows is = {}"
      .format(users[(users['occupation'].isnull()) & (users['salary'].isnull())].shape[0]))
print("The following are the first 5 rows that have been replaced:\n{}"
      .format(users[(users['occupation'].isnull()) & (users['salary'].isnull())].head()))

In [ ]:
# Let's go back to the original occupation and salary
users = users.fillna({'occupation': 'none'})
users['salary'] = users['occupation'].map(lambda o: occupation_to_salary[o])

## Discretization and Binning

-  Continuous data is often **discretized** or otherwised separated into "**bins**" for analysis. 

-  The typical example is given by the case where you have user's data containing information like 'age', and you what to divide users in a set of fixed age intervals.

In [ ]:
"""
Let's consider our 'age' column and suppose we want to divide these into bins, 
such as users aged between 0 and 17, 18 to 25, 26 to 35, 
36 to 55, and finally 56 and older. To do so, we can use pandas.cut function.
"""
# Let's first define a list containing the left-most extreme of each bin
bins = [0, 18, 26, 36, 56, 100]
"""
Consistent with mathematical notation for intervals, a parenthesis ')' 
means that the side is OPEN while the square bracket '[' means it is closed (inclusive). 
By default, intervals are left-open, i.e., (a, b]
Passing 'right=False' those become right-open, i.e., [a, b) 
"""
age_intervals = pd.cut(users['age'], bins, right=False)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

In [ ]:
"""
The object pandas returns is a special Categorical object. 
We can treat it like an array of strings indicating the bin name; 
internally it contains a categories array indicating the distinct category names 
along with a labeling for the ages data in the 'codes' attribute
"""
print("Categorical bin codes: {}".format(age_intervals.cat.codes.head()))
print("Categorical bin names: {}".format(age_intervals.cat.categories))

In [ ]:
"""
Instead of integer labeling for the bin, we can specify which label
to assign to each bin.
"""
age_labels = ['Young', 'Young_Adult', 'Adult', 'Middle_Aged', 'Senior']

age_intervals = pd.cut(users['age'], bins, labels=age_labels, right=False)

print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

In [ ]:
"""
Let's create an extra column 'age_interval' on the DataFrame with this information.
"""
users['age_interval'] = pd.cut(users['age'], bins, labels=age_labels, right=False)
print(users.head())

In [ ]:
"""
Sometimes we don't want to specify the intervals ourselves, instead
we want to just specify the number of bins and let pandas figure out how 
data are distributed across those bins.
"""
# If we pass 'cut' an integer number of bins instead of explicit bin edges, 
# it will compute equal-length bins based on the minimum and maximum values in the data. 
age_intervals = pd.cut(users['age'], 5, right=False)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

In [ ]:
# The same as above, but with labels
age_labels = ['Young', 'Young_Adult', 'Adult', 'Middle_Aged', 'Senior']
age_intervals = pd.cut(users['age'], 5, labels=age_labels, right=False)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

In [ ]:
"""
A closely related function 'qcut' bins the data based on sample quantiles. 
Depending on the distribution of the data, using 'cut' will not usually result 
in each bin having the same number of data points. 
Instead, with 'qcut' by definition we will obtain roughly equal-size bins.
"""
# We are using quartiles here [0.25, 0.50, 0.75, 1]
# Alternatively, we can specify the list of our own quantiles 
# (i.e., numbers between 0 and 1, inclusive)
age_intervals = pd.qcut(users['age'], 4)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

In [ ]:
# The same as above yet with labels
age_labels = ['Young', 'Adult', 'Middle_Aged', 'Senior']
age_intervals = pd.qcut(users['age'], 4, labels=age_labels)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

## Detecting and Filtering Outliers

In [ ]:
"""
Let's see the output of the 'describe()' function on our DataFrame.
"""
print(users.describe(include='all'))

In [ ]:
"""
Suppose we want to see what are the records where 'salary' is greater than 280k.
"""
salary_outlier = users.salary > 280000
print("There are {} salary outliers, 5 of which are as follows:\n{}"
      .format(users[salary_outlier].shape[0], users[salary_outlier].head()))

In [ ]:
"""
If we want to change the value of an outlier with a 'cap' value (e.g., 280000)
"""
# 1. Using 'where'
users.salary = users.salary.where(~salary_outlier, 280000)
print(users[salary_outlier].head())

In [ ]:
"""
Let's revert back to the original salary
"""
users.salary = users.salary.where(~salary_outlier, 300000)
print(users[salary_outlier].head())

In [ ]:
"""
If we want to change the value of an outlier with a 'cap' value (e.g., 280000)
"""
# 2. Using 'loc'
users.loc[salary_outlier, 'salary'] = 280000
print(users[salary_outlier].head())

## Combining <code>pandas</code> Objects

-  Data contained in <code>**pandas**</code> objects (i.e., <code>**Series**</code> and <code>**DataFrame**</code>) can be combined together in a number of built-in ways, such as:

    -  <code>**pandas.merge**</code>/<code>**pandas.join**</code> connects rows of two objects based on one or more keys. This is equivalent to **join** operations on relational databases.

    -  <code>**pandas.concat**</code> concatenates or "stacks" together objects along a specific axis, if any (there is only a single possible axis of concatenation for <code>**Series**</code>).

-  Full API documentation is available [here](https://pandas.pydata.org/pandas-docs/stable/merging.html)

## Database-style <code>DataFrame</code> Joins

In [ ]:
"""
Merge or join operations combine data sets by linking rows using one or more keys. 
These operations are central to relational databases (e.g., SQL-based). 
The 'merge' function in pandas is the main entry point for achieving this.
"""
# Let's go back to our original DataFrame
del users['salary']
del users['age_interval']
print(users.head())

## What is a Join/Merge Operation?

-  Generally speaking, "joining" ("merging") two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

-  Typical operation in relational databases using <code>**SQL JOIN**</code> operator.

-  In <code>**pandas**</code> there are two different functions <code>**merge**</code> and <code>**join**</code>, both of which do similar things; the former is used for column-to-column joins, whereas the latter is more efficient when joining <code>**DataFrame**</code> objects on their indices.

## The Anatomy of a Join/Merge Operation

```python
merged_df = pd.merge(left_df, right_df, 
                     left_on=["l_col_1",...,"l_col_n"], 
                     right_on=["r_col_1",...,"r_col_n"],
                     how="{left|right|inner|outer}"
```

## The Anatomy of a Join/Merge Operation

-  <code>**left_df**</code> and <code>**right_df**</code> are the two <code>**DataFrame**</code> objects we want to merge.

-  <code>**left_on**</code> and <code>**right_on**</code> indicate the column(s) used for the merging operation on the left and right <code>**DataFrame**</code> objects, respectively (alternatively, use just <code>**on=[col_1,...,col_n]**</code> if column names are the same on both <code>**DataFrame**</code>s).

-  <code>**how**</code> is used to specify which kind of merge operation needs to be performed, i.e., one of: <code>**left**</code>, <code>**right**</code>, <code>**inner**</code> (default), and <code>**outer**</code>.

## Different Types of Join/Merge Operations

(./img/join_types.jpg)

<center>[Image Source](https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/)</center>

-  **Inner Merge/Inner Join** (default): Keep only those rows where the value which we want to merge on exists in **both** the **left** and **right** <code>**DataFrame**</code>s.

-  **Left Merge/Left (Outer) Join**: Keep every row in the **left** <code>**DataFrame**</code>. If some values of the column we are merging on are missing in the **right** <code>**DataFrame**</code>, add <code>**NaN**</code> to the result.

-  **Right Merge/Right (Outer) Join**: Keep every row in the **right** <code>**DataFrame**</code>. If some values of the column we are merging on are missing in the **left** <code>**DataFrame**</code>, add <code>**NaN**</code> to the result.

-  **Outer Merge/Full (Outer) Join**: Return **all** the rows from the **left** and the **right** <code>**DataFrame**</code>s, matches up rows where possible, otherwise add <code>**NaN**</code>.

In [ ]:
# Suppose we have another DataFrame containing the (average) salary for some occupations. 
# Let's assume we don't have salary information for 'technician' and 'other'
# but we do for two extra occupations (e.g., 'gardener' and 'professor')
occupation_salary_data = {'occupation': ['administrator', 'writer', 'executive', 
                                       'student', 'lawyer', 'educator', 
                                       'scientist', 'entertainment', 
                                       'programmer', 'librarian', 'homemaker', 
                                       'artist', 'engineer', 'marketing', 
                                       'none', 'healthcare', 'retired', 
                                       'salesman', 'doctor', 'gardener', 
                                       'professor'],
        'salary': [150000, 40000, 300000, 1300, 27500, 45000, 
                      60000, 185000, 55000, 22000, 240000, 72000, 
                      91000, 66000, 0, 41000, 52000, 48000, 
                      140000, 16000, 82000]}

occupation_salary = pd.DataFrame(occupation_salary_data)
print("Occupation-Salary data:\n{}".format(occupation_salary.head()))

In [ ]:
users

In [ ]:
"""
By default, 'merge' tries to join DataFrames on the basis of common column names.
In our example there is a column called 'occupation', which is common to both DataFrames.
Moreover, 'merge' implements an INNER JOIN, which means that the resulting DataFrame 
will contain all and only those records which match on both the left and right DataFrame. 
"""
merged = pd.merge(users, occupation_salary)
# The above is equivalent to:
# merged = users.merge(occupation_salary)
# Or:
# merged = pd.merge(users, occupation_salary, 
#                   left_on="occupation", right_on="occupation", how="inner")
# Or, again:
# merged = pd.merge(users, occupation_salary, on="occupation", how="inner")

In [ ]:
merged

In [ ]:
print("Number of records in the left DataFrame: {}"
      .format(users.shape[0]))
print("Number of records in the right DataFrame: {}"
      .format(occupation_salary.shape[0]))
print("Unique values of 'occupation' in the left DataFrame:\n{}".
      format(users.occupation.unique()))
print("Unique values of 'occupation' in the right DataFrame:\n{}".
      format(occupation_salary.occupation.unique()))

In [ ]:
print("Number of records in the resulting merged DataFrame: {}"
      .format(merged.shape[0]))
print(merged.head())

In [ ]:
"""
Double check if the number of resulting rows after (inner) merge is compliant with
what we expect. Remember, inner merge results in a number of records which correspond
to the intersection of values on which we merge the 2 DataFrames on.
"""
# Let's see how many occupation values in the left DataFrame (users) 
# appear also in the right DataFrame (occupation_salary)
print("Number of 'occupation' values shared between the two DataFrames:\n{}"
      .format())

In [ ]:
"""
Double check if the number of resulting rows after (inner) merge is compliant with
what we expect. Remember, inner merge results in a number of records which correspond
to the intersection of values on which we merge the 2 DataFrames on.
"""
# Let's see how many occupation values in the left DataFrame (users) 
# appear also in the right DataFrame (occupation_salary)
print("Number of 'occupation' values shared between the two DataFrames:\n{}"
      .format(users.occupation.isin(occupation_salary.occupation).value_counts()))

In [ ]:
"""
Suppose we want to keep the records from the left DataFrame
"""
merged_left = pd.merge(users, occupation_salary, how='left')
print(merged_left.head())

In [ ]:
"""
How many records do we expect the resulting merged DataFrame above to have?
"""
print("Actual number of records: {}".format(merged_left.shape[0]))

In [ ]:
"""
Suppose we want to keep the records from the right DataFrame
"""
merged_right = pd.merge(users, occupation_salary, how='right')
print(merged_right.head())

In [ ]:
"""
How many records do we expect the resulting merged DataFrame above to have?
"""
print("Actual number of records: {}".format(merged_right.shape[0]))

In [ ]:
"""
When we use 'merge' on column(s)-to-column(s) 
the indexes associated with the merging DataFrame objects are discarded.
"""
# Suppose we want to re-assign an Index object to the new merged DataFrame
# using the previous non-default index of the left DataFrame (users)
merged.index.name = 'user_id'

merged.index = users.index[users.occupation.isin(occupation_salary.occupation)]
print(merged.head())

In [ ]:
# Suppose we want to create another DataFrame containing the (average) salary
# for a set of 100 random users (i.e., not occupations)
# 1. Let's pick 100 user_ids uniformly at random
np.random.seed(23)

random_users = np.random.choice(users.index, 100, replace=False)

print("Selected number of random users = {}"
      .format(random_users.size))

print("Q: Is there any duplicates? A: {}"
      .format(pd.Series(random_users).duplicated().any()))

# print("Q: Is there any duplicates? A: {}"
#       .format())

In [ ]:
# 2. Let's do the same with the average salary.
# This time we extract a 100-sample drawn from a Normal distribution.
# To do so, we need to specify the two parameters of the distribution: 
# mean (mu) and standard deviation (sigma)
mu = 35000
sigma = 10000
normal_salaries = np.random.normal(mu, sigma, 100)

# Let's also round salaries to the 2nd decimal digit
normal_salaries = np.round(normal_salaries, decimals=2)
print(normal_salaries)

In [ ]:
# 2. Let's do the same with the average salary.
# This time we extract a 100-sample drawn from a Normal distribution.
# To do so, we need to specify the two parameters of the distribution: 
# mean (mu) and standard deviation (sigma)
mu = 35000
sigma = 10000
normal_salaries = np.random.normal(loc=mu, scale=sigma, size=100)
# Let's also round salaries to the 2nd decimal digit
normal_salaries = np.round(normal_salaries, decimals=2)
# Alternatively:
# np.round(normal_salaries, decimals=2, out=normal_salaries)

In [ ]:
# 3. Let's now create the dictionary containing two keys: 'user_id' and 'salary'
# For each key, we associate the list of random_users and normal_salaries, respectively
user_to_salary_data = {'user_id': random_users, 'salary': normal_salaries}

# 4. We create the corresponding pandas DataFrame object
user_to_salary = pd.DataFrame(user_to_salary_data)

user_to_salary.set_index('user_id', inplace=True)

print("User-Salary data:\n{}".format(user_to_salary.head()))

In [ ]:
# 5. Finally, merge (join) the original DataFrame (users) with this new one
# Merging uses the index this time, rather than the column as before
merged = pd.merge(users, user_to_salary, left_index=True, right_index=True)

# Note that if you don't specify that you are merging on indexes, 
# merge raises an error!
print(merged.head())

In [ ]:
# 5. Finally, merge (join) the original DataFrame (users) with this new one
# Merging uses the index this time, rather than the column as before
#merged = pd.merge(users, user_to_salary, left_index=True, right_index=True)
# Note that if you don't specify that you are merging on indexes, 
# merge raises an error!
# This is equivalent to (join is 'left' by default):

merged = users.join(user_to_salary, how='inner')

print(merged.head())

## <code>pandas.merge</code> Arguments (1 of 2)

<p align="center">
  <img src="./img/pd_merge_args_1.png">
</p>

## <code>pandas.merge</code> Arguments (1 of 2)

<p align="center">
  <img src="./img/pd_merge_args_2.png">
</p>

## Options for the <code>how</code> Argument

<p align="center">
  <img src="./img/pd_merge_how.png">
</p>

## Concatenating <code>DataFrame</code>s

In [ ]:
"""
We can use the 'concat' function in pandas to append either rows or columns (if any) 
from one object to another. 
Let's grab two subsets of our DataFrame to see how this works.
"""
# Read in first 5 rows of users table
users_first_5 = users.head()
print("First 5 records of 'users':\n{}".format(users_first_5))
print()
# Read in the last 5 rows
users_last_5 = users[-6:]
print("Last 5 records of 'users':\n{}".format(users_last_5))

In [ ]:
"""
We can use the 'concat' function in pandas to append either rows or columns (if any) 
from one object to another. 
Let's grab two subsets of our DataFrame to see how this works.
"""
# Read in first 5 rows of users table
users_first_5 = users.head()
print("First 5 records of 'users':\n{}".format(users_first_5))
print()
# Read in the last 5 rows
users_last_5 = users[-5:]
print("Last 5 records of 'users':\n{}".format(users_last_5))

## Stack "_vertically_" vs. "_horizontally_": <code>axis</code>

-  By default, <code>**concat**</code> operates on <code>**axis=0**</code> (i.e., **rows**) and tells <code>**pandas**</code> to stack the second DataFrame under the first one (i.e., **vertically**). 
    -  In order for this to work, we need to make sure that both <code>**DataFrame**</code>s have the same column names and formats. 

-  Instead, <code>**axis=1**</code> will stack the **columns** in the second <code>**DataFrame**</code> to the right of the first one (i.e., **horizontally**). 
    - In this case, we want to make sure that the data we stack are related in some way.

In [ ]:
"""
Stack DataFrames vertically (i.e., using default axis=0)
"""
# Stack the two DataFrames 'users_first_5' and 'users_last_5' on top of each other
vertical_stack = pd.concat([users_first_5, users_last_5], axis=0)
print(vertical_stack)

In [ ]:
"""
Stack DataFrames horizontally (i.e., using axis=1)
"""
# Place the two DataFrames 'users_first_5' and 'users_last_5' side by side
horizontal_stack = pd.concat([users_first_5, users_last_5], axis=1)
print(horizontal_stack)

In [ ]:
"""
Since the row indexes for the two DataFrames 'users_first_5' and 'users_last_5' 
are not the same, 'concat' can't place them next to each other (NaN values occur). 
A workaround for this is to reindex our second DataFrame using the 'reset_index()' method.
"""
# Set the index of the second DataFrame to that of the first one
users_last_5.set_index(users_first_5.index, inplace=True)
horizontal_stack = pd.concat([users_first_5, users_last_5], axis=1)
print(horizontal_stack)

In [ ]:
"""
Since the row indexes for the two DataFrames 'users_first_5' and 'users_last_5' 
are not the same, 'concat' can't place them next to each other (NaN values occur). 
A workaround for this is to reindex our second DataFrame using the 'reset_index()' method.
"""
# Set the index of the second DataFrame to that of the first one
users_last_5.set_index(users_first_5.index, inplace=True)
# Or:
# users_last_5 = users_last_5.set_index(users_first_5.index)
horizontal_stack = pd.concat([users_first_5, users_last_5], axis=1)
print(horizontal_stack)

In [ ]:
horizontal_stack['age']

## <code>pandas.concat</code> Arguments

<p align="center">
  <img src="./img/pd_concat.png">
</p>
